Poniższy kod służył demonstracji przeuczenia modeli używających danych standaryzowanych.
Modele zostały jednak prawdopodobnie dotknięte poprzez dokonywanie przewidywań bez użycia torch.no_grad(), co spowodowało że wartości R2 dla obu zbiorów wyrównały się

In [1]:
import os
import import_ipynb
import torch
import data_acquisition

import model_creator

from single_file_prediction import decode_model

from scipy.stats import pearsonr

def create_and_load(model_folder,model_name, input_size, output_size):
    model_path = os.path.join(model_folder, model_name)
    model = model_creator.MLP(input_size, output_size, hidden_layers=decode_model(model_name))
    if torch.cuda.is_available():
        model.load_state_dict(torch.load(model_path))
    else:
        model.load_state_dict(torch.load(model_path, map_location=torch.device('cpu')))
    return model

def list_to_float(lst):
    return [float(i) for i in lst]

parent_folder = os.getcwd()
print(parent_folder)
folder_path = os.path.join(parent_folder,"datasets", "new_Si_jaw_delta", "")
print(folder_path)
model_folder = os.path.join(parent_folder,"models" ,"")
print(model_folder)
half_data_folder = os.path.join(parent_folder,"datasets", "half_new_Si_jaw_delta", "")
quarter_data_folder = os.path.join(parent_folder,"datasets", "quarter_new_Si_jaw_delta", "")





Adata = data_acquisition.getData(["wavelength","psi65", "del65", "psi70", "del70", "psi75", "del75"], ["A"], folder_path)
Bdata = data_acquisition.getData(["wavelength","psi65", "del65", "psi70", "del70", "psi75", "del75"], ["B"], folder_path)
Cdata = data_acquisition.getData(["wavelength","psi65", "del65", "psi70", "del70", "psi75", "del75"], ["C"], folder_path)
Csdata = data_acquisition.get_Standarized_data("cScaler",["wavelength","psi65", "del65", "psi70", "del70", "psi75", "del75"], ["C"], folder_path)
Bsdata = data_acquisition.get_Standarized_data("bScaler",["wavelength","psi65", "del65", "psi70", "del70", "psi75", "del75"], ["B"], folder_path)








def calculate_set_r2(model, x,y):
    model.eval()
    with torch.no_grad():
        pred = model(x)
        pred = pred.flatten().tolist()
    y= y.flatten().tolist()
    return (pearsonr(pred, y)[0]**2)

def calculate_r2(model, data):
    model.eval()
    with torch.no_grad():
        predTrain = model(data[0])
        predTrain = predTrain.flatten().tolist()
        predTest = model(data[2])
        predTest = predTest.flatten().tolist()
        yTrain = data[1].flatten().tolist()
        yTest = data[3].flatten().tolist()
    return ([pearsonr(predTrain, yTrain)[0]**2, pearsonr(predTest, yTest)[0]**2])

A_32_16_16_8 = create_and_load(model_folder, "modelA_32_16_16_8.pth", 7, 1)
print(f"modelA_32_16_16_8 R2 Train: {calculate_r2(A_32_16_16_8, Adata)[0]} Test: {calculate_r2(A_32_16_16_8, Adata)[1]}")
A_48_32_24_12 = create_and_load(model_folder, "modelA_48_32_24_12.pth", 7, 1)
print(f"modelA_48_32_24_12 R2 Train: {calculate_r2(A_48_32_24_12, Adata)[0]} Test: {calculate_r2(A_48_32_24_12, Adata)[1]}")
A_64_32_32_16 = create_and_load(model_folder, "modelA_64_32_32_16.pth", 7, 1)
print(f"modelA_64_32_32_16 R2 Train: {calculate_r2(A_64_32_32_16, Adata)[0]} Test: {calculate_r2(A_64_32_32_16, Adata)[1]}")
B_32_16_16_8 = create_and_load(model_folder, "modelB_32_16_16_8.pth", 7, 1)
print(f"modelB_32_16_16_8 R2 Train: {calculate_r2(B_32_16_16_8, Bdata)[0]} Test: {calculate_r2(B_32_16_16_8, Bdata)[1]}")
B_48_32_24_12 = create_and_load(model_folder, "modelB_48_32_24_12.pth", 7, 1)
print(f"modelB_48_32_24_12 R2 Train: {calculate_r2(B_48_32_24_12, Bdata)[0]} Test: {calculate_r2(B_48_32_24_12, Bdata)[1]}")
B_64_32_32_16 = create_and_load(model_folder, "modelB_64_32_32_16.pth", 7, 1)
print(f"modelB_64_32_32_16 R2 Train: {calculate_r2(B_64_32_32_16, Bdata)[0]} Test: {calculate_r2(B_64_32_32_16, Bdata)[1]}")
C_32_16_16_8 = create_and_load(model_folder, "modelC_32_16_16_8.pth", 7, 1)
print(f"modelC_32_16_16_8 R2 Train: {calculate_r2(C_32_16_16_8, Cdata)[0]} Test: {calculate_r2(C_32_16_16_8, Cdata)[1]}")
C_48_32_24_12 = create_and_load(model_folder, "modelC_48_32_24_12.pth", 7, 1)
print(f"modelC_48_32_24_12 R2 Train: {calculate_r2(C_48_32_24_12, Cdata)[0]} Test: {calculate_r2(C_48_32_24_12, Cdata)[1]}")
C_64_32_32_16 = create_and_load(model_folder, "modelC_64_32_32_16.pth", 7, 1)
print(f"modelC_64_32_32_16 R2 Train: {calculate_r2(C_64_32_32_16, Cdata)[0]} Test: {calculate_r2(C_64_32_32_16, Cdata)[1]}")

Cst_32_16_16_8 = create_and_load(model_folder, "modelCstandard_32_16_16_8.pth", 7, 1)
print(f"modelCstandard_32_16_16_8 R2 Train: {calculate_r2(Cst_32_16_16_8, Csdata)[0]} Test: {calculate_r2(Cst_32_16_16_8, Csdata)[1]}")
Cst_48_32_24_12 = create_and_load(model_folder, "modelCstandard_48_32_24_12.pth", 7, 1)
print(f"modelCstandard_48_32_24_12 R2 Train: {calculate_r2(Cst_48_32_24_12, Csdata)[0]} Test: {calculate_r2(Cst_48_32_24_12, Csdata)[1]}")
Cst_64_32_32_16 = create_and_load(model_folder, "modelCstandard_64_32_32_16.pth", 7, 1)
print(f"modelCstandard_64_32_32_16 R2 Train: {calculate_r2(Cst_64_32_32_16, Csdata)[0]} Test: {calculate_r2(Cst_64_32_32_16, Csdata)[1]}")
Bst_32_16_16_8 = create_and_load(model_folder, "modelBstandard_32_16_16_8.pth", 7, 1)
print(f"modelBstandard_32_16_16_8 R2 Train: {calculate_r2(Bst_32_16_16_8, Bsdata)[0]} Test: {calculate_r2(Bst_32_16_16_8, Bsdata)[1]}")
Bst_48_32_24_12 = create_and_load(model_folder, "modelBstandard_48_32_24_12.pth", 7, 1)
print(f"modelBstandard_48_32_24_12 R2 Train: {calculate_r2(Bst_48_32_24_12, Bsdata)[0]} Test: {calculate_r2(Bst_48_32_24_12, Bsdata)[1]}")
Bst_64_32_32_16 = create_and_load(model_folder, "modelBstandard_64_32_32_16.pth", 7, 1)
print(f"modelBstandard_64_32_32_16 R2 Train: {calculate_r2(Bst_64_32_32_16, Bsdata)[0]} Test: {calculate_r2(Bst_64_32_32_16, Bsdata)[1]}")

csXdatacombined = torch.cat((Csdata[0], Csdata[2]), dim=0)
csYdatacombined = torch.cat((Csdata[1], Csdata[3]), dim=0)
bsXdatacombined = torch.cat((Bsdata[0], Bsdata[2]), dim=0)
bsYdatacombined = torch.cat((Bsdata[1], Bsdata[3]), dim=0)

print(f"combined modelCstandard_64_32_32_16 R2: {calculate_set_r2(Cst_64_32_32_16, csXdatacombined, csYdatacombined)}")
print(f"combined modelBstandard_64_32_32_16 R2: {calculate_set_r2(Bst_64_32_32_16, bsXdatacombined, bsYdatacombined)}")
print(f"combined modelCstandard_48_32_24_12 R2: {calculate_set_r2(Cst_48_32_24_12, csXdatacombined, csYdatacombined)}")
print(f"combined modelBstandard_48_32_24_12 R2: {calculate_set_r2(Bst_48_32_24_12, bsXdatacombined, bsYdatacombined)}")
print(f"combined modelCstandard_32_16_16_8 R2: {calculate_set_r2(Cst_32_16_16_8, csXdatacombined, csYdatacombined)}")
print(f"combined modelBstandard_32_16_16_8 R2: {calculate_set_r2(Bst_32_16_16_8, bsXdatacombined, bsYdatacombined)}")



cshalfdata = data_acquisition.get_Standarized_data("cScaler",["wavelength","psi65", "del65", "psi70", "del70", "psi75", "del75"], ["C"], half_data_folder)
bshalfdata = data_acquisition.get_Standarized_data("bScaler",["wavelength","psi65", "del65", "psi70", "del70", "psi75", "del75"], ["B"], half_data_folder)


c_half_64_32_32_16 = create_and_load(model_folder, "modelhalfCstandard_64_32_32_16.pth", 7, 1)
print(f"modelhalfCstandard_64_32_32_16 R2 Train: {calculate_r2(c_half_64_32_32_16, cshalfdata)[0]} Test: {calculate_r2(c_half_64_32_32_16, cshalfdata)[1]}")
b_half_64_32_32_16 = create_and_load(model_folder, "modelhalfBstandard_64_32_32_16.pth", 7, 1)
print(f"modelhalfBstandard_64_32_32_16 R2 Train: {calculate_r2(b_half_64_32_32_16, bshalfdata)[0]} Test: {calculate_r2(b_half_64_32_32_16, bshalfdata)[1]}")

cs_halfX_data_combined = torch.cat((cshalfdata[0], cshalfdata[2]), dim=0)
cs_halfY_data_combined = torch.cat((cshalfdata[1], cshalfdata[3]), dim=0)
bs_halfX_data_combined = torch.cat((bshalfdata[0], bshalfdata[2]), dim=0)
bs_halfY_data_combined = torch.cat((bshalfdata[1], bshalfdata[3]), dim=0)

print(f"combined modelhalfCstandard_64_32_32_16 R2: {calculate_set_r2(c_half_64_32_32_16, cs_halfX_data_combined, cs_halfY_data_combined)}")
print(f"combined modelhalfBstandard_64_32_32_16 R2: {calculate_set_r2(b_half_64_32_32_16, bs_halfX_data_combined, bs_halfY_data_combined)}")



cquarterdata = data_acquisition.get_Standarized_data("cScaler",["wavelength","psi65", "del65", "psi70", "del70", "psi75", "del75"], ["C"], quarter_data_folder)
bquarterdata = data_acquisition.get_Standarized_data("bScaler",["wavelength","psi65", "del65", "psi70", "del70", "psi75", "del75"], ["B"], quarter_data_folder)


c_quarter_64_32_32_16 = create_and_load(model_folder, "modelquarterCstandard_64_32_32_16.pth", 7, 1)
print(f"modelquarterCstandard_64_32_32_16 R2 Train: {calculate_r2(c_quarter_64_32_32_16, cquarterdata)[0]} Test: {calculate_r2(c_quarter_64_32_32_16, cquarterdata)[1]}")

b_quarter_64_32_32_16 = create_and_load(model_folder, "modelquarterBstandard_64_32_32_16.pth", 7, 1)
print(f"modelquarterBstandard_64_32_32_16 R2 Train: {calculate_r2(b_quarter_64_32_32_16, bquarterdata)[0]} Test: {calculate_r2(b_quarter_64_32_32_16, bquarterdata)[1]}")


cs_halfX_data_combined = torch.cat((cquarterdata[0], cquarterdata[2]), dim=0)
cs_halfY_data_combined = torch.cat((cquarterdata[1], cquarterdata[3]), dim=0)
bs_halfX_data_combined = torch.cat((bquarterdata[0], bquarterdata[2]), dim=0)
bs_halfY_data_combined = torch.cat((bquarterdata[1], bquarterdata[3]), dim=0)
print(f"combined modelquarterCstandard_64_32_32_16 R2: {calculate_set_r2(c_quarter_64_32_32_16, cs_halfX_data_combined, cs_halfY_data_combined)}")
print(f"combined modelquarterBstandard_64_32_32_16 R2: {calculate_set_r2(b_quarter_64_32_32_16, bs_halfX_data_combined, bs_halfY_data_combined)}")


D:\ML\machineLearningEllipsometry\code_data_models
D:\ML\machineLearningEllipsometry\code_data_models\datasets\new_Si_jaw_delta\
D:\ML\machineLearningEllipsometry\code_data_models\models\
Index(['wavelength', 'psi65', 'del65', 'psi70', 'del70', 'psi75', 'del75', 'T',
       'A', 'B', 'C'],
      dtype='object')
Index(['wavelength', 'psi65', 'del65', 'psi70', 'del70', 'psi75', 'del75', 'T',
       'A', 'B', 'C'],
      dtype='object')
Index(['wavelength', 'psi65', 'del65', 'psi70', 'del70', 'psi75', 'del75', 'T',
       'A', 'B', 'C'],
      dtype='object')
modelA_32_16_16_8 R2 Train: 0.8525147138922837 Test: 0.851606708156159
modelA_48_32_24_12 R2 Train: 0.8671922424288933 Test: 0.866597724841283
modelA_64_32_32_16 R2 Train: 0.0024750041028505053 Test: 0.0021088461207320807
modelB_32_16_16_8 R2 Train: 0.009854404040679625 Test: 0.008501113030604548
modelB_48_32_24_12 R2 Train: 0.013914532489162242 Test: 0.012993214341879077
modelB_64_32_32_16 R2 Train: 0.0002509130241640421 Test: 0.000

C:\Users\Public\Documents\Wondershare\CreatorTemp\ipykernel_26004\4112206687.py:67: ConstantInputWarning: An input array is constant; the correlation coefficient is not defined.
  return ([pearsonr(predTrain, yTrain)[0]**2, pearsonr(predTest, yTest)[0]**2])


modelCstandard_32_16_16_8 R2 Train: 0.00019838551005110888 Test: 0.00043510620277890667
modelCstandard_48_32_24_12 R2 Train: 8.56511176695042e-05 Test: 0.0002579439176620367
modelCstandard_64_32_32_16 R2 Train: 1.0362974892595486e-05 Test: 0.00016036546787843428
modelBstandard_32_16_16_8 R2 Train: 0.07824148145668182 Test: 0.0789716149442441
modelBstandard_48_32_24_12 R2 Train: 0.0009059253000736848 Test: 0.0004967474029865426
modelBstandard_64_32_32_16 R2 Train: 0.0013327738038497554 Test: 0.0008679540647029771
combined modelCstandard_64_32_32_16 R2: 2.6081760047536423e-05
combined modelBstandard_64_32_32_16 R2: 0.0012317894459179053
combined modelCstandard_48_32_24_12 R2: 0.00011392042553233446
combined modelBstandard_48_32_24_12 R2: 0.0008143336224887227
combined modelCstandard_32_16_16_8 R2: 0.00023870157881444236
combined modelBstandard_32_16_16_8 R2: 0.07838656273937654
modelhalfCstandard_64_32_32_16 R2 Train: 0.0011019021996312882 Test: 0.0008093491897491592
modelhalfBstandard_6